In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 # GloVe Bidirectional LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

# panggil dataset
df = pd.read_csv('Whatsapp.csv')
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,"['respond', 'message', 'android', 'auto', 'ack...",negative
1,1,"['I', 've', 'use', 'app', 'nearly', 'decade', ...",positive
2,2,"['daily', 'spam', 'text', 'call', 'get', 'app'...",positive
3,3,"['problem', 'perfection', 'make', 'change', 'd...",positive
4,4,"['good', 'interface', 'nice', 'functionality',...",positive


In [ ]:
# menggabungkan token menjadi 1 kalimat
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: ' '.join(eval(x)))
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,negative
1,1,I ve use app nearly decade though new useful f...,positive
2,2,daily spam text call get app take put two two ...,positive
3,3,problem perfection make change downgrade do no...,positive
4,4,good interface nice functionality need wish wo...,positive


In [ ]:
# mengubah label menjadi format numerik
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,0
1,1,I ve use app nearly decade though new useful f...,2
2,2,daily spam text call get app take put two two ...,2
3,3,problem perfection make change downgrade do no...,2
4,4,good interface nice functionality need wish wo...,2


In [ ]:
# membagi data
X = df['text_lemmatized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(88090,) (9788,) (88090,) (9788,)


In [ ]:
# panggil file GloVe
glove_file = '/content/drive/MyDrive/Dicoding/ProjectAnalisisSentimen/Glove/glove.6B.100d.txt'

# membaca embedding GloVe
embeddings_index = {}
with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Total {len(embeddings_index)} word vectors.')

Total 400000 word vectors.


In [ ]:
# menggunakan tokenizer untuk teks
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# mengonversi teks menjadi urutan indeks kata
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# padding sequences
maxlen = 184 # menyesuaikan maksimal panjang pada teks
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen)

# menyimpan informasi kata
word_index = tokenizer.word_index
print(f'ditemukan {len(word_index)} tokens yang unik / unique .')

ditemukan 42496 tokens yang unik / unique .


In [ ]:
# membuat matriks embedding
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(f'Embedding matrix shape: {embedding_matrix.shape}')


Embedding matrix shape: (42497, 100)


In [ ]:
# menghitung class weights untuk ketidakseimbangan kelas
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

# mengubah label menjadi one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# membangun model LSTM
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

# setup optimier, loss, dan metrics untuk mengukur akurasi
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 184, 100)          4249700   
                                                                 
 bidirectional (Bidirection  (None, 184, 256)          234496    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 184, 256)          0         
                                                                 
 batch_normalization (Batch  (None, 184, 256)          1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                        

In [ ]:
# membuat callbacks
checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')

# melatih model
model.fit(X_train_padded, y_train_one_hot, epochs=50, batch_size=32, validation_split=0.2,
          class_weight=class_weights_dict, callbacks=[checkpoint, early_stopping])

Epoch 1/50
2203/2203 [==============================] - 90s 37ms/step - loss: 0.6378 - accuracy: 0.7262 - val_loss: 0.3807 - val_accuracy: 0.8566
Epoch 2/50
2203/2203 [==============================] - 82s 37ms/step - loss: 0.3971 - accuracy: 0.8480 - val_loss: 0.3029 - val_accuracy: 0.8958
Epoch 3/50
2203/2203 [==============================] - 76s 35ms/step - loss: 0.3059 - accuracy: 0.8845 - val_loss: 0.3031 - val_accuracy: 0.8945
Epoch 4/50
2203/2203 [==============================] - 78s 35ms/step - loss: 0.2549 - accuracy: 0.9047 - val_loss: 0.1984 - val_accuracy: 0.9350
Epoch 5/50
2203/2203 [==============================] - 77s 35ms/step - loss: 0.2223 - accuracy: 0.9177 - val_loss: 0.1802 - val_accuracy: 0.9405
Epoch 6/50
2203/2203 [==============================] - 76s 35ms/step - loss: 0.1979 - accuracy: 0.9254 - val_loss: 0.1765 - val_accuracy: 0.9363
Epoch 7/50
2203/2203 [==============================] - 77s 35ms/step - loss: 0.1842 - accuracy: 0.9324 - val_loss: 0.1539 -

In [ ]:
# panggil model terbaik
best_model = load_model('best_lstm_model.keras')

# memprediksi dengan data latih
y_train_pred = best_model.predict(X_train_padded)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_classes = np.argmax(y_train_one_hot, axis=1)

print("Data Latih:")
print(classification_report(y_train_classes, y_train_pred_classes, target_names=label_encoder.classes_))

2753/2753 [==============================] - 35s 12ms/step
Data Latih:
              precision    recall  f1-score   support

    negative       0.94      0.95      0.95     24581
     neutral       0.95      0.98      0.96     12466
    positive       0.98      0.97      0.97     51043

    accuracy                           0.97     88090
   macro avg       0.96      0.97      0.96     88090
weighted avg       0.97      0.97      0.97     88090



In [ ]:
# memprediksi dengan data uji
y_test_pred = best_model.predict(X_test_padded)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_classes = np.argmax(y_test_one_hot, axis=1)

print("Data Uji:")
print(classification_report(y_test_classes, y_test_pred_classes, target_names=label_encoder.classes_))

306/306 [==============================] - 3s 11ms/step
Data Uji:
              precision    recall  f1-score   support

    negative       0.92      0.94      0.93      2693
     neutral       0.94      0.96      0.95      1375
    positive       0.97      0.96      0.97      5720

    accuracy                           0.96      9788
   macro avg       0.95      0.95      0.95      9788
weighted avg       0.96      0.96      0.96      9788



In [ ]:
def preprocess_and_predict(text, tokenizer, maxlen, model, label_encoder):
    # preprocessing teks yang diinput
    tokens = tokenizer.texts_to_sequences([text])
    tokens_padded = pad_sequences(tokens, maxlen=maxlen)

    # prediksi
    prediction = model.predict(tokens_padded)
    predicted_class = np.argmax(prediction, axis=1)

    # mengubah kelas prediksi menjadi label asli
    predicted_label = label_encoder.inverse_transform(predicted_class)
    return predicted_label[0]

# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: really worst
1/1 [==============================] - 0s 83ms/step
teks ini masuk ke : negative


In [ ]:
# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: great
1/1 [==============================] - 0s 19ms/step
teks ini masuk ke : positive


In [ ]:
# input dari user
user_input = input("Input Teks: ")
predicted_sentiment = preprocess_and_predict(user_input, tokenizer, maxlen, best_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: standart
1/1 [==============================] - 0s 27ms/step
teks ini masuk ke : neutral
